# Modelling most important features

## Model Selection

In [1]:
# Importing the libraries
import os
import pandas as pd

## Data Preprocessing

In [2]:
# Load and preprocess the train data 
from pipelines import preprocessing_pipeline

data_file = os.path.join('..', '..', '..', '..', 'data', 'raw', 'train.csv')
train_data = pd.read_csv(data_file, index_col=0, low_memory=False)
train_data = preprocessing_pipeline.fit_transform(train_data)
train_data.head()

,day_phase,bg-5:45,bg-5:30,bg-5:15,bg-5:00,bg-4:45,bg-4:30,bg-4:15,bg-4:00,bg-3:45,...,insulin-2:00,insulin-1:45,insulin-1:30,insulin-1:15,insulin-1:00,insulin-0:45,insulin-0:30,insulin-0:15,insulin-0:00,bg+1:00
id,,,,,,,,,,,,,,,,,,,,,
p01_0,morning,9.6,9.7,9.2,8.7,8.4,8.1,8.3,9.6,11.1,...,0.0583,0.0583,0.0583,0.058300,0.058300,0.058300,0.058300,0.052767,0.0417,13.4
p01_1,morning,9.7,9.2,8.7,8.4,8.1,8.3,9.6,11.1,11.8,...,0.0583,0.0583,0.0583,0.058300,0.058300,0.058300,0.052767,0.041700,0.0417,12.8
p01_2,morning,9.2,8.7,8.4,8.1,8.3,9.6,11.1,11.8,12.8,...,0.0583,0.0583,0.0583,0.058300,0.058300,0.052767,0.041700,0.041700,0.0417,15.5
p01_3,morning,8.7,8.4,8.1,8.3,9.6,11.1,11.8,12.8,13.9,...,0.0583,0.0583,0.0583,0.058300,0.052767,0.041700,0.041700,0.041700,0.0417,14.8
p01_4,morning,8.4,8.1,8.3,9.6,11.1,11.8,12.8,13.9,14.2,...,0.0583,0.0583,0.0583,0.052767,0.041700,0.041700,0.041700,0.041700,0.0417,12.7


## Data Splitting

In [3]:
from sklearn.model_selection import train_test_split

X = train_data.drop(columns=['bg+1:00'])
y = train_data['bg+1:00']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Standardization

In [4]:
from pipelines import standardization_pipeline

X_train = standardization_pipeline.fit_transform(X_train)
X_test = standardization_pipeline.transform(X_test)

X_train.head()

,bg-5:45,bg-5:30,bg-5:15,bg-5:00,bg-4:45,bg-4:30,bg-4:15,bg-4:00,bg-3:45,bg-3:30,...,insulin-1:00,insulin-0:45,insulin-0:30,insulin-0:15,insulin-0:00,day_phase_evening,day_phase_late_evening,day_phase_morning,day_phase_night,day_phase_noon
id,,,,,,,,,,,,,,,,,,,,,
p12_24040,-0.493555,-0.493993,-0.527083,-0.693575,-0.859838,-1.026946,-0.993637,-1.127547,-1.093278,-0.758704,...,-0.239877,-0.341920,-0.341050,-0.343226,-0.074042,0,0,0,0,1
p12_13267,-0.028365,-0.061718,-0.127596,-0.427020,-0.326456,-0.359978,-0.426541,-0.292884,-0.192100,-0.091437,...,-0.167386,-0.001651,-0.124055,-0.343226,-0.198222,0,0,1,0,0
p03_131,0.769103,0.869334,1.037576,1.238950,1.473706,1.474186,1.208032,0.875644,1.042847,1.243097,...,0.283511,10.116378,0.112904,-0.008536,-0.007780,0,0,0,0,0
p12_20861,0.569736,0.935838,0.871123,0.672520,0.306934,0.607127,0.907804,1.042576,1.142978,1.343188,...,-0.341365,1.964986,-0.341050,-0.285029,-0.198222,0,0,0,0,0
p12_24627,-0.991972,-1.059275,-0.859989,-0.726895,-0.593147,-0.460023,-0.126313,0.174527,-0.158723,-0.758704,...,-0.341365,-0.125647,-0.341050,-0.343226,-0.198222,0,0,0,0,0


## Model Selection with LazyPredict

In [5]:
from notebooks.helpers.LazyPredict import get_lazy_regressor

reg = get_lazy_regressor(exclude=['SVR'])
models, predictions = reg.fit(X_train, X_test, y_train, y_test)
models

 97%|█████████▋| 37/38 [12:52<00:07,  7.56s/it] 

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.006099 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 12211
[LightGBM] [Info] Number of data points in the train set: 141619, number of used features: 53
[LightGBM] [Info] Start training from score 8.276012


100%|██████████| 38/38 [12:54<00:00, 20.37s/it]


,Adjusted R-Squared,R-Squared,RMSE,Time Taken
Model,,,,
ExtraTreesRegressor,0.73,0.73,1.55,123.82
BaggingRegressor,0.67,0.67,1.74,43.18
XGBRegressor,0.63,0.63,1.82,1.33
KNeighborsRegressor,0.60,0.60,1.90,3.95
LGBMRegressor,0.59,0.59,1.93,2.09
HistGradientBoostingRegressor,0.59,0.59,1.93,2.64
MLPRegressor,0.58,0.58,1.95,42.54
GradientBoostingRegressor,0.54,0.54,2.03,87.63
LassoLarsCV,0.51,0.51,2.10,0.63


### We select the best models with RMSE < 2.0

In [6]:
models[models['RMSE'] < 2.0]

,Adjusted R-Squared,R-Squared,RMSE,Time Taken
Model,,,,
ExtraTreesRegressor,0.73,0.73,1.55,123.82
BaggingRegressor,0.67,0.67,1.74,43.18
XGBRegressor,0.63,0.63,1.82,1.33
KNeighborsRegressor,0.60,0.60,1.90,3.95
LGBMRegressor,0.59,0.59,1.93,2.09
HistGradientBoostingRegressor,0.59,0.59,1.93,2.64
MLPRegressor,0.58,0.58,1.95,42.54
